# Problem 9
Estimate the critical radius of a hypothetical bare spherical reactor consisting of a liquid mixture of sodium and plutonium in which the plutonium is present to a 3.0 w/o and the mixture density is approximately 1.0 g/cc.

## Model
Here we create the materials, geometry, and settings comprising a model of our reactor.  

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import openmc
import openmc.model

%matplotlib inline

In [3]:
def build_model(radius):
    core = openmc.Material(name='core');
    core.add_element('Na',0.97,'wo');
    core.add_nuclide('Pu239',0.03,'wo');
    core.set_density('g/cm3',1.0);

    materials = openmc.Materials([core]);
    
    R = radius;
    core_g = openmc.Sphere(r=R,boundary_type='vacuum');


    core_c = openmc.Cell();
    core_c.fill = core;
    core_c.region = -core_g;
    core_c.temperature = 900;

    root_u = openmc.Universe();
    root_u.add_cells([core_c]);

    geometry = openmc.Geometry(root_u);
    
    settings = openmc.Settings();
    settings.batches = 300;
    settings.inactive = 100;
    settings.particles = 20000;

    bounds = [-R,-R,-R,R,R,R];
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                       only_fissionable=True);
    settings.source = openmc.source.Source(space=uniform_dist);

    settings.temperature['method']='interpolation';
    settings.output = {'tallies': False};
    
    model = openmc.model.Model(geometry,materials,settings);
    
    return model
    
    

## Search for Critical Radius
I have the solution manual and I think the critical radius is around 400 cm but this is a well-behavied sort of root-finding problem so there is no real issue with establishing wide bounds.

In [4]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model,
                                               bracket=[50.,1000.],
                                               tol=1e-3,
                                               print_iterations=True);

print(f'Critical radius: %5.4f cm' % crit_r);

Iteration: 1; Guess of 5.00e+01 produced a keff of 0.03179 +/- 0.00003
Iteration: 2; Guess of 1.00e+03 produced a keff of 1.49898 +/- 0.00043
Iteration: 3; Guess of 5.25e+02 produced a keff of 1.37261 +/- 0.00044
Iteration: 4; Guess of 2.88e+02 produced a keff of 1.05077 +/- 0.00053
Iteration: 5; Guess of 1.69e+02 produced a keff of 0.56450 +/- 0.00041
Iteration: 6; Guess of 2.28e+02 produced a keff of 0.85414 +/- 0.00046
Iteration: 7; Guess of 2.58e+02 produced a keff of 0.96367 +/- 0.00050
Iteration: 8; Guess of 2.73e+02 produced a keff of 1.00927 +/- 0.00047
Iteration: 9; Guess of 2.65e+02 produced a keff of 0.98703 +/- 0.00050
Iteration: 10; Guess of 2.69e+02 produced a keff of 0.99802 +/- 0.00045
Iteration: 11; Guess of 2.71e+02 produced a keff of 1.00373 +/- 0.00048
Iteration: 12; Guess of 2.70e+02 produced a keff of 1.00104 +/- 0.00049
Iteration: 13; Guess of 2.69e+02 produced a keff of 0.99953 +/- 0.00047
Iteration: 14; Guess of 2.70e+02 produced a keff of 1.00117 +/- 0.00047
C

Interesting.  The "solution" presented for problem 9 results in a critical radius of 405 cm.  One thing that I can do is to verify that I've (at least) specified the materials correctly.  In the blocks below I will inspect the nuclide densities and compare to what the "solution" has computed.

In [6]:
core = openmc.Material(name='core');
core.add_element('Na',0.97,'wo');
core.add_nuclide('Pu239',0.03,'wo');
core.set_density('g/cm3',1.0);

nuclides = core.get_nuclide_atom_densities();

print(nuclides['Na23']);
print(nuclides['Pu239']);


('Na23', 0.025409026770295454)
('Pu239', 7.55752319697297e-05)


this closely corresponds to the computed atom densities of 2.54e-2 and 7.56e-5 atoms/b-cm respectively.